In [1]:
using JLD
include("gyncycle.jl");
parms, y0 = loadparms()
data = loadpfizer();

In [ ]:
SIGMA_RHO = 0.1
SIGMA_Y0 = 10
SIGMA_PARMS = 10
SIGMA_PROPOSAL = 0.05
adapt = :all
stepsize = 1000
sampler = :amm
persons = 1:3
samplingnodes = [:sparms, :y0]

jldname = "s$sampler a$adapt p$(length(persons))-r$SIGMA_RHO-sy$SIGMA_Y0-spar$SIGMA_PARMS-spro$SIGMA_PROPOSAL-2truncflatprior.jld"

d = eye(length(SAMPLEPARMS)+length(y0))
proposalvariance = d * log(1+(SIGMA_PROPOSAL^2));

samplers = Dict(
    :amm=>[AMM(samplingnodes, proposalvariance, adapt=adapt)], 
    :amwg=>[AMWG(samplingnodes,diag(proposalvariance); target=0.44)])

mcs = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, samplers[:amwg])
    
    push!(mcs, mcmc(m, inp, ini, stepsize, burnin=0, verbose = false, thin = 1))
end

#samplelength(mc) = max(floor(Int, size(mc, 1)/5), 2_000)

println(jldname)
while size(mcs[1],1) < 1_000_000
    @time for i in 1:length(persons)
        mcs[persons[i]] = mcmc(mcs[persons[i]], stepsize, verbose=false)
    end
    @time save(jldname, "mc", [mc.value for mc in mcs])    
    println(size(mcs[1]))
end

In [6]:
mc=(mcs[1])

Object of type "Mamba.ModelChains"

Iterations = 1:6800
Thinning interval = 1
Chains = 1
Samples per chain = 6800

6800x115x1 Array{Float64,3}:
[:, :, 1] =
  7309.9    7309.9   192.2    2.371    …  0.0009368    3.3169e-5   2.9879e-5 
  7309.9    7309.9   192.2    2.371       0.0009368    3.3169e-5   2.9879e-5 
  7494.67   7291.75  185.33   2.15389     0.000954463  3.1256e-5   2.94678e-5
  7494.67   7291.75  185.33   2.15389     0.000954463  3.1256e-5   2.94678e-5
  7494.67   7291.75  185.33   2.15389     0.000954463  3.1256e-5   2.94678e-5
  8470.53   7111.81  174.029  1.94567  …  0.000823409  2.73357e-5  2.70335e-5
  8470.53   7111.81  174.029  1.94567     0.000823409  2.73357e-5  2.70335e-5
  8470.53   7111.81  174.029  1.94567     0.000823409  2.73357e-5  2.70335e-5
  8470.53   7111.81  174.029  1.94567     0.000823409  2.73357e-5  2.70335e-5
  8470.53   7111.81  174.029  1.94567     0.000823409  2.73357e-5  2.70335e-5
  8470.53   7111.81  174.029  1.94567  …  0.000823409  2.73357e-

In [8]:
s=mc.model.samplers[1]

An object of type "Mamba.Sampler"
Sampling Block Nodes:
[:sparms,:y0]

AST(:($(Expr(:lambda, Any[:(model::Model),:(block::Integer)], Any[Any[Any[:model,:Any,18],Any[:block,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        block = (top(typeassert))(block,Mamba.Integer)
        f = (anonymous function)
        return f(model,block)
    end)))))


In [13]:
st=s.tune["sampler"]

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:


Mamba.AMMTune(true,0.05,6800,[-1.48885,-1.77653,-2.3699,-3.08047,-1.71877,-2.80893,-3.16564,-2.6272,-0.854527,-1.77569  …  -2.35358,-2.27553,-1.95803,-3.0306,-1.1344,-1.45472,-3.14488,-3.09168,-3.0158,-1.94529],115x115 Array{Float64,2}:
 2.31235  2.64702  3.48719   4.52272  2.56753  …   4.61347   4.44184  2.90985
 2.64702  3.26274  4.24937   5.47535  3.0782       5.43263   5.29576  3.47328
 3.48719  4.24937  5.69742   7.36994  4.09656      7.32838   7.16416  4.63736
 4.52272  5.47535  7.36994   9.61848  5.3279       9.57253   9.35907  6.01711
 2.56753  3.0782   4.09656   5.3279   2.99892      5.32742   5.18127  3.37067
 4.10412  4.84576  6.59814   8.65627  4.7549   …   8.69625   8.58029  5.40459
 4.65421  5.59777  7.52111   9.78901  5.41827      9.80726   9.59332  6.14187
 3.89553  4.63555  6.22906   8.13117  4.52247      8.14845   7.96282  5.11093
 1.33644  1.61928  2.07264   2.63054  1.53742      2.64103   2.47265  1.72594
 2.64298  3.24764  4.2466    5.45564  3.06436      5.4596    

In [21]:
cond(st.SigmaLm)

424.804520251625